### Imports

In [1]:
import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(19)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

from keras import backend as K
K.set_image_dim_ordering('th')
from keras.utils import np_utils
from sklearn.metrics import f1_score

%matplotlib inline

/home/whale/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load MNIST

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

### Preprocessing

In [3]:
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

In [4]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

### Model

In [5]:
def baseline_model():
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss = keras.losses.categorical_crossentropy, 
                  optimizer = keras.optimizers.Adadelta(), 
                  metrics=['accuracy'])
    return model

In [18]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 200, verbose = 2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose = 0)
print("Baseline Error: %.2f%%" % (100 - scores[1] * 100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 231s - loss: 0.6793 - acc: 0.7856 - val_loss: 0.0959 - val_acc: 0.9731
Epoch 2/10
 - 224s - loss: 0.1951 - acc: 0.9480 - val_loss: 0.0672 - val_acc: 0.9815
Epoch 3/10
 - 226s - loss: 0.1441 - acc: 0.9625 - val_loss: 0.0619 - val_acc: 0.9826
Epoch 4/10
 - 223s - loss: 0.1191 - acc: 0.9693 - val_loss: 0.0537 - val_acc: 0.9856
Epoch 5/10
 - 228s - loss: 0.1027 - acc: 0.9740 - val_loss: 0.0459 - val_acc: 0.9866
Epoch 6/10
 - 239s - loss: 0.0913 - acc: 0.9757 - val_loss: 0.0431 - val_acc: 0.9884
Epoch 7/10
 - 233s - loss: 0.0844 - acc: 0.9777 - val_loss: 0.0563 - val_acc: 0.9852
Epoch 8/10
 - 242s - loss: 0.0770 - acc: 0.9796 - val_loss: 0.0516 - val_acc: 0.9874
Epoch 9/10
 - 255s - loss: 0.0720 - acc: 0.9817 - val_loss: 0.0467 - val_acc: 0.9894
Epoch 10/10
 - 231s - loss: 0.0673 - acc: 0.9829 - val_loss: 0.0382 - val_acc: 0.9900
Baseline Error: 1.00%


### Score

In [6]:
model = baseline_model()
model.fit(X_train, y_train,
          batch_size=512,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 139s 2ms/step - loss: 0.6555 - acc: 0.7867 - val_loss: 0.1776 - val_acc: 0.9479
Epoch 2/10
60000/60000 [==============================] - 127s 2ms/step - loss: 0.1389 - acc: 0.9568 - val_loss: 0.0808 - val_acc: 0.9736
Epoch 3/10
60000/60000 [==============================] - 147s 2ms/step - loss: 0.0981 - acc: 0.9694 - val_loss: 0.0651 - val_acc: 0.9804
Epoch 4/10
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0785 - acc: 0.9751 - val_loss: 0.0477 - val_acc: 0.9844
Epoch 5/10
60000/60000 [==============================] - 160s 3ms/step - loss: 0.0645 - acc: 0.9794 - val_loss: 0.0442 - val_acc: 0.9857
Epoch 6/10
60000/60000 [==============================] - 178s 3ms/step - loss: 0.0566 - acc: 0.9824 - val_loss: 0.0443 - val_acc: 0.9848
Epoch 7/10
60000/60000 [==============================] - 166s 3ms/step - loss: 0.0517 - acc: 0.9835 - val_loss: 0.0508 - 

In [7]:
scores = model.evaluate(X_test, y_test, verbose=0, )
print(f'loss: {scores[0]: 0.5f}')
print(f'accuracy:{scores[1]*100: 0.2f}%')

loss:  0.02740
accuracy: 99.11%
